In [2]:
import sys
import os
sys.path.append("../")

In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np
import plotly_template as pt
from functions import emissions
import definitions
import pandas as pd
import pickle
import math
import numpy as np

# pio.renderers.default = "browser"

# Generating Iterations of the Heatmaps

### Creating the emissions plot function

### File locations and constants

In [17]:
DATA_FOLDER = 'Route_Sampler_Mainline_Only_Turn_Ratio_Included'
DATA_DIR = r'Z:\6 ACTION Project\SUMO\Sim_Output_Data'
# DATA_DIR_FULL_PATH = os.path.join(definitions.DATA_DIR, DATA_FOLDER)
DATA_DIR_FULL_PATH = os.path.join(DATA_DIR, DATA_FOLDER)
RAW_DATA_FILE = 'data.csv'
RAW_DATA_SUMMARY = 'data_summary.csv'

### Bining the Emissions Data

In [18]:
resample_period = '15T' # aka the bin size
bin_size = 5

In [19]:
df = pd.read_csv(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE), low_memory=False, index_col=0)
df['timestep_time'] = pd.to_datetime(df['timestep_time'])
emissions.bin_2D_sum(emissions_df=df, bin_column=['vehicle_fuel'], resample_period=resample_period, bin_size=5, processor_num=2)

c:\users\spraychamber\us69-research-paper-plotting\venv\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



func:bin_2D_sum took: 5.22148323059082 sec


(None, 5.22148323059082)

### Grabbing the Data

In [20]:
binned_emissions_dict = pickle.load(open(os.path.join(definitions.DATA_DIR, 'emissions_dict.pkl'), 'rb'))

### Plotting the Data

#### Plot Constants

In [21]:
plot_radius = 10
mapbox_key = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"

colorscale = [
              [0.0, "rgba(0, 255, 204, 0)"],
              [0.2, "rgb(0, 255, 51)"],
              [0.4, "rgb(204, 255, 0)"],
              [0.6, "rgb(255, 204, 51)"],
              [0.8, "rgb(255, 102, 51)"],
              [1.0, "rgb(204,0,0)"],
            ]

color_axis = dict(
            cmin=0,
            cmax=None,
            showscale=True,
            colorscale=colorscale,
            colorbar=dict(
                outlinecolor="black",
                outlinewidth=2,
                ticks="outside",
                # tickfont=pt.font_dict,
                tickvals=None, # tickvalues,
                ticktext=None,
                title="[gal]<br /> <br />",
                tickformat="e",
                # titlefont=pt.font_dict,
            )
)

In [25]:
plot_time_list = [pd.to_datetime('2020-02-13T07:30:00'), pd.to_datetime('2020-02-13T10:00:00'), pd.to_datetime('2020-02-13T16:00:00')]

num_plots = len(plot_time_list)
fig = make_subplots(rows=1, cols=num_plots,
                    specs=[[{'type': 'mapbox', }] * num_plots],
                    subplot_titles=[str(time) for time in plot_time_list],
                    column_widths=[(1 - 0.1 * (num_plots / 2)) / num_plots]*num_plots,
                    horizontal_spacing=0.05,)

mapbox_dict = {'mapbox': dict(accesstoken=mapbox_key,
                                bearing=0,
                                style='mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte',
                                center=go.layout.mapbox.Center(
                                    lat=33.126275,
                                    lon=-87.54891
                                ),
                                pitch=0,
                                zoom=14.45),}

max_value = -20
for i, time in enumerate(plot_time_list):
    
    z_data = binned_emissions_dict[time]['data']
    lat_edges = binned_emissions_dict[time]['lat']
    lon_edges = binned_emissions_dict[time]['lon']
    max_value = binned_emissions_dict[time]['max_value'] if binned_emissions_dict[time]['max_value'] > max_value else max_value
    
    fig.add_trace(go.Densitymapbox(lat=lat_edges,
                                    lon=lon_edges,
                                    z=z_data,
                                    # customdata=summed_array[0][2],
                                    # hovertemplate='%{customdata}' if sum else None,
                                    radius=plot_radius,
                                    hoverinfo='z',
                                    coloraxis="coloraxis",
                                    ),
                    row=1,
                    col=i+1)
    
    mapbox_dict['mapbox'+ str(i+2)] =  mapbox_dict['mapbox']
    
color_axis['cmax'] = math.ceil(max_value * 1000) / 1000

len_of_max = len(str(color_axis['cmax']))
# tickvalues = np.around(np.linspace(0, np.round(color_axis['cmax'], len_of_max-1), 7), 3)
tickvalues = np.around(np.linspace(0, np.round(color_axis['cmax'], len_of_max), 7), 3)
ticktext = [str(val) for val in tickvalues]

color_axis['colorbar']['tickvals'] = tickvalues
color_axis['colorbar']['ticktext'] = ticktext

In [10]:
max_value

0.10073406703999996

In [110]:
[(1 - 0.1 * (num_plots / 2)) / num_plots]*num_plots

[0.2833333333333333, 0.2833333333333333, 0.2833333333333333]

In [23]:
color_axis

{'cmin': 0,
 'cmax': 0.138,
 'showscale': True,
 'colorscale': [[0.0, 'rgba(0, 255, 204, 0)'],
  [0.2, 'rgb(0, 255, 51)'],
  [0.4, 'rgb(204, 255, 0)'],
  [0.6, 'rgb(255, 204, 51)'],
  [0.8, 'rgb(255, 102, 51)'],
  [1.0, 'rgb(204,0,0)']],
 'colorbar': {'outlinecolor': 'black',
  'outlinewidth': 2,
  'ticks': 'outside',
  'tickvals': array([0.   , 0.023, 0.046, 0.069, 0.092, 0.115, 0.138]),
  'ticktext': ['0.0', '0.023', '0.046', '0.069', '0.092', '0.115', '0.138'],
  'title': '[gal]<br /> <br />',
  'tickformat': 'e'}}

In [26]:
fig.update_layout(mapbox_dict)

fig.update_layout(
    title={"text": "Fuel Consumption", 'yanchor': 'middle', 'xanchor': 'center'},
    hovermode='closest',
#     mapbox=dict(
#         accesstoken=mapbox_key,
#         bearing=0,
#         style='mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte',
#         center=go.layout.mapbox.Center(
#             lat=33.12627,
#             lon=-87.54891
#         ),
#         pitch=0,
#         zoom=14.1,
#     ),
    coloraxis=color_axis
)

fig.show('browser')
pio.write_json(fig, os.path.join(definitions.ROOT, 'raw_plots', 'mapbox.plotly'))